## ***Uncertainty Quantification***

Exercise 03, 
June 10,2021


Prof. Dr. M.Frank


Dr. Jonas Kusch


Pia Stammer


Maqsood Rajput

-----------------------------------------------------------------------------------------

Note: To render Jupyter Notebook and to run code on Bw-cloud instance, please sign up at https://e5188803.ka.bw-cloud-instance.org/ 

-----------------------------------------------------------------------------------------

#### **EXERCISE SHEET 3**

In [ ]:
!pip install chaospy
import numpy as np
import math
import scipy.special
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
from matplotlib import cm
import chaospy.distributions.copulas.clayton

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

**EXERCISE 1)**

Consider the ODE with uncertainties
$$
\dot u(t) = -z_1 u(t),\quad u(0)=z_2.
$$ 
Derive expressions for the
sensitivities of 
$$R(u) = \int_0^T u(t)dt$$ 
with respect to $z_1$ and $z_2$ by

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) using the explicit solution,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) using forward differentiation,

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) using adjoints.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

**EXERCISE 2)**

Look at the code below. The Clayton copula can be
stated as

$$
C(u_1 ,u_2 )= \max ((u_1^{-\theta} +u_2^{-\theta} -1)^{-1/\theta},0),\quad  \theta > 0
$$

Sklar's Theorem says that any joint distribution function $F(x_1,x_2)$
with strictly increasing marginal distribution functions $F_1(x_1)$ and
$F_2(x_2)$ may be written as

$$
F (x_1, x_2) = C(u_1, u_2) \text{ where } u_1 = F_1(x_1), u_2 = F_2(x_2).
$$

Additionally, the copula may be combined with either of

$$
\begin{aligned}
&u_1 = F_1(x_1), 1 - u_2 = F_2(x_2)\quad &\text{ orientation (1,2)}\\
&1 - u_1 = F_1(x_1), u_2 = F_2(x_2)\quad &\text{ orientation (2,1)}\\
&1 - u_1 = F_1(x_1), 1 - u_2 = F_2(x_2)\quad &\text{ orientation (2,2)}
\end{aligned}
$$

and the effect is to rotate the copula patterns compared to the original
one.

Let the random variables $X_1$ and $X_2$ be distributed as
$\mathcal{N}(\mu=0.005, \sigma^2 = 0.05^2)$. Construct a two-dimensional
joint distribution by using the Clayton copula with different values of
$\theta$ and different orientations. Sample from the joint cdf and
visualize by using the given code.

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

def FInv(U,mu,sigma):
    return scipy.special.erfinv(2*U-1)*math.sqrt(2)*sigma+mu

#choose theta value
theta = 0.5          # Change this value and see how the distribution changes

#plot corresponding copula C(u1,u2)
u = np.linspace(1,0,10,endpoint=True)
u1,u2 = np.meshgrid(u,u)
grid = np.empty((2,u.size**2))
grid[1,:]= u1.flatten()
grid[0,:]= u2.flatten()


#change orientation 
#
# add code to change the orientation of the Copula to (2,1),(1,2) or (2,2)
#


dist = dist = chaospy.Iid(chaospy.Uniform(), 2)
clayton_cop = chaospy.Clayton(dist,theta)
y = clayton_cop.pdf(grid)

y = np.reshape(y,(10,10))
fig = plt.figure(figsize=(10,6))
ax = fig.gca(projection='3d')
surf = ax.plot_surface(u1, u2, y, cmap=cm.viridis, linewidth=0, antialiased=False)

#sample u1, u2 from C
u = clayton_cop.sample(100)

#plot histogram of u1,u2
plt.figure()
scatter_axes = plt.subplot2grid((3, 3), (1, 0), rowspan=2, colspan=2)
plt.subplots_adjust(wspace=0.5,hspace=0.5)
x_hist_axes = plt.subplot2grid((3, 3), (0, 0), colspan=2,
                               sharex=scatter_axes)
y_hist_axes = plt.subplot2grid((3, 3), (1, 2), rowspan=2,
                               sharey=scatter_axes)

scatter_axes.plot(u[0,:],u[1,:], '.')
_ = x_hist_axes.hist(u[0,:],5,color = "white", ec="C0")
_ = y_hist_axes.hist(u[1,:],5, orientation='horizontal',color = "white", ec="C0")

#transform to Gaussian variables
mu = 0.005;
sigma = 0.05;
x1 = FInv(u[0,:],mu,sigma);
x2 = FInv(u[1,:],mu,sigma);

#plot histogram of x1,x2
plt.figure()
scatter_axes = plt.subplot2grid((3, 3), (1, 0), rowspan=2, colspan=2) 
plt.subplots_adjust(wspace=0.5,hspace=0.5)
x_hist_axes = plt.subplot2grid((3, 3), (0, 0), colspan=2,
                               sharex=scatter_axes)
y_hist_axes = plt.subplot2grid((3, 3), (1, 2), rowspan=2,
                               sharey=scatter_axes)

scatter_axes.plot(x1, x2, '.')
_ = x_hist_axes.hist(x1,color = "white", ec="C0")
_ = y_hist_axes.hist(x2, orientation='horizontal',color = "white", ec="C0")


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

**EXERCISE 3)**

Prove the Nataf covariance formula 

$$
\begin{aligned}
\text{cov}(Z_i,Z_j) = \int\int (z_i-\mu_i)(z_j-\mu_j)\rho_i(z_i)\rho_j(z_j)\frac{\varphi_2(y_i(z_i),y_j(z_j))}{\varphi_1(y_i(z_i))\varphi_1(y_j(z_j))}\,dz_i\,dz_j,
\end{aligned}
$$

where $\varphi_1$ is a standard normal pdf and $\varphi_2$ is the
bivariate normal pdf with covariance matrix 

$$
\begin{aligned}
V = \begin{pmatrix}
1 & V_{ij} \\
V_{ji} & 1
\end{pmatrix}.\end{aligned}
$$ 

**Hint:** Use
$\left(f^{-1}\right)'(a) = \frac{1}{f'(f^{-1}(a))}$.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

**EXERCISE 4)**

The KL expansion of the random process $Z_t$ is given by

\begin{equation*}
Z_t(\omega)=\sum_{j=1}^{\infty}\sqrt{\lambda_j}\psi_j(t)Z_j(\omega)
\end{equation*}

where

\begin{equation*}
Z_j(\omega)=\frac{1}{\sqrt{\lambda_j}}\left(Z_t{(\omega)},\psi_j(t)\right)=\frac{1}{\sqrt{\lambda_j}}\int_0^TZ_t(\omega)\psi_j(t)dt.
\end{equation*}

Show that if E$[Z_t]=0$, then E$[Z_i]=0$, E$[Z_iZ_j]=\delta_{ij}$. This means that the new variables $Z_i$ are uncorrelated.


&nbsp;

### **Additional Exercises**

&nbsp;

**EXERCISE 5)**

We want to compute the expected value of $g(Z)=\sqrt{Z} sin(2\pi Z)$, where $Z$ is uniformly distributed in [0,1], with the help of Monte-Carlo. We use control variates to speed up the computation. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) Plot $g$ and the control variate $\lambda$ by running the following block of code. Choose different values of $\lambda$ and try to get 



In [ ]:
#Define g and h

def g(Z):
  y=math.sqrt(Z)*math.sin(2*math.pi*Z)
  return y

def h(Z):
  y=math.sin(2*math.pi*Z)
  return y 

g=np.vectorize(g)
h=np.vectorize(h)

#Plot g and control variate h
lambda_c = 10
zGrid = np.linspace(start=0,stop=1,num=1000)
plt.figure(figsize=(8,5))
plt.plot(zGrid,g(zGrid),label='g')
plt.plot(zGrid,lambda_c*h(zGrid),linestyle='--',linewidth=2,label='h')
_=plt.legend(shadow=True, fancybox=True)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) Compute the expected value of $g$ by using Monte Carlo with and without control variate. How does the choice of $\lambda$ affect the solution?

In [ ]:
N=5000  #number of samples

tmpMC = 0
tmpCV = 0

expectedMC = np.zeros((N,1))
expectedCV = np.zeros((N,1))

for i in range(N):
  Z=np.random.uniform(0,1,(1,1))

  #Your code: Save MC and CV expectation value approximation with i+1 samples in expectedMC(i) and expectedCV(i)
  #...
  #expectedMC[i], expectedCV[i], tmpMC, tmpCV = solution_b(tmpMC,tmpCV,Z,lambda_c,i+1) #(run code section below exercise c) & uncomment this line to see solution)

print("Expected value is {0}".format(expectedMC[N-1]))

expectedExact = -0.11980826271647446

plt.figure(figsize=(8,5))
plt.loglog(np.arange(1,N+1),abs(expectedMC-expectedExact),label='MC')
plt.loglog(np.arange(1,N+1),abs(expectedCV-expectedExact),label='CV')
plt.loglog(np.arange(1,N+1),1/np.sqrt(np.arange(1,N+1)),linestyle='--',linewidth=2,label='slope 1/2')
_=plt.legend(shadow=True, fancybox=True)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) Look at the code below. What is implemented here? How is the value of $\lambda$ picked?

In [ ]:
def estimateCov (covOld,QgOld,N,Z):
  y=(N-1)/N*covOld + 1/N* (g(Z)-QgOld)* (h(Z)-QgOld)
  return y

#Compute integral with CV and optimal lambda

expectedCVL = np.zeros((N,1))
tmpCVL = 0
covOld = 0.5           #estimated covariance
expectedOld = -0.1     #estimated mean
varH = 0.5             #exact variance of function h

for i in range(N):
  Z = np.random.uniform(0,1,(1,1))
  cov = estimateCov(covOld, expectedOld, i+1 , Z) #update covariance
  lambdaOpt = cov/varH
  tmpCVL = tmpCVL + (g(Z) - lambdaOpt*h(Z))
  expectedCVL[i] = 1/(i+1)*tmpCVL
  covOld = cov
  expectedOld = expectedCVL[i]

print("Expected value is {0}".format(expectedCVL[N-1]))

expectedExact = -0.11980826271647446
plt.figure(figsize=(8,5))
plt.loglog(np.arange(1,N+1),abs(expectedMC-expectedExact),label='MC')
plt.loglog(np.arange(1,N+1),abs(expectedCV-expectedExact),linestyle='--',linewidth=2,label='CV')
plt.loglog(np.arange(1,N+1),abs(expectedCVL-expectedExact),linestyle='-.',linewidth=2,label='CVOpt')
plt.loglog(np.arange(1,N+1),1/np.sqrt(np.arange(1,N+1)),label='slope 1/2')
_=plt.legend(shadow=True, fancybox=True)

print("Lambda is {0}".format(lambdaOpt))

plt.figure(figsize=(8,5))
plt.plot(zGrid,g(zGrid),label='g')
plt.plot(zGrid,np.squeeze(lambdaOpt*h(zGrid)),label='$\lambda_{Opt} \; h$')
plt.plot(zGrid,lambda_c*h(zGrid),linestyle='--',linewidth=2,label='$\lambda \; h$')
_=plt.legend(shadow=True, fancybox=True)

In [ ]:
#Solution to fill in field in (b)

def solution_b (tmpMC,tmpCV,Z,lambda_c,i):
  tmpMC = tmpMC + g(Z);
  tmpCV = tmpCV + (g(Z)-lambda_c*h(Z));
  expectedMC_i = (1.0/i)*tmpMC;
  expectedCV_i = (1.0/i)*tmpCV;

  return expectedMC_i, expectedCV_i, tmpMC, tmpCV


&nbsp;

**EXERCISE 6)**

Assume a (discretized) system can be written in the form
$$ E(u,z) = 0,$$

where
- the solution $u$ $\in \mathbb{R}^K$
- the parameters $z$ $\in \mathbb{R}^N$

and the constraint E: $\mathbb{R}^K \times \mathbb{R}^N \rightarrow \mathbb{R}^K$ admits a unique solution for $u$ depending on $z$. Assume further that we have quantities of interest
$$R(u) = R(u(z))$$
with $R: \mathbb{R}^K \rightarrow \mathbb{R}^I$. We are interested in the sensitivity of $R$ with respect to $z$, i.e.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) Derive formulas for the sensitivities by forward-differentiation.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) Derive formulas for the sensitivities by adjoint calculus.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) Discuss the dimensions of all objects. When is forward differentiation advantageous, when adjoints?

&nbsp;


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

**EXERCISE 7)**

A bivariate copula is the joint distribution function

$$
C(u_1,u_2)=P(U_1 \leq u_1,U_2 \leq u_2),\quad 0<u_1 \leq 1, 0< u_2 \leq 1.
$$

Any function $C(u_1,u_2)$ that is to play this role must be increasing
in $u_1$ and $u_2$ and satisfy $C(u_1,0) = 0$, $C(0,u_2) = 0$ and
$C(u_1,1) = u_1$, $C(1,u_2) = u_2$.

Introduce the copulas

$$
C_{\min}(u_1, u_2) = {\max}(u_1 + u_2  -1, 0) \text{ and } C_{\max}(u_1, u_2) = {\min}(u_1, u_2),
$$

where $0 < u_1, u_2 < 1$.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (a) Argue that $C_{\min}(u_1, u_2)$ is the copula when $U_2 = 1 - U_1$.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (b) Also argue that $C_{\max}(u_1,u_2)$ corresponds to $U_2 = U_1$.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   (c) Verify that any copula $C(u_1,u_2)$ satisfies

$$C_{\min}(u_1, u_2) \leq C(u_1, u_2) \leq C_{\max}(u_1, u_2), 0 \leq u_1, u_2 \leq 1.$$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   This is known as the Frechet-Hoeffding inequality and shows that
$C_{\min}(u_1, u_2)$ is a minimum and $C_{\max}(u_1, u_2)$ a maximum copula.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   

*Hint:* For the upper bound note that
- $C(u_1,u_2) \leq P(U_i \leq u_i)$ and for the lower one introduce
- $H(u_1)=C(u_1,u_2)-(u_1+u_2-1)$ for which $H(1)=0$ and
- $\frac{dH(u_1)}{du_1} = P(U_2 \leq u_2 | u_1)-1\leq 0$.